# StructuredOutputParser
StructuredOutputParser은 LLM에 대한 답변을 `dict`형식으로 구성하고, key/value쌍으로 여러 필드를 반환하고자 할때 사용한다. 구조화된 형식을 dict형식으로 받을 수 있다.

보통 Pydantic/JSON에서 더 강력하다는 평가를 받지만, 보통 GPT, Claude모델에 대해서 강력하다.

로컬모델과 같이 인텔리전스가 적은 즉, 파라미터가 적은 모델에 효과적이다. 

local model을 사용할 경우, Pydantic파서가 동작하지 않을 수 있는데, Local model은 Pydantic 문법들을 알아듣지 못할 수 있기 때문이다.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_teddynote import logging
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [6]:
# 사용자 질문에 대한 답변 양식. local model이기 때문에 description은 왠만하면 영어로 적는게 좋다.
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 '출처','웹사이트주소' 이여야 합니다."
    )
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

 pydantic의 경우
```
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
```

In [8]:
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 '출처','웹사이트주소' 이여야 합니다.
}
```


In [17]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template = "answer the users question as best as possible. \n{format_instructions}\n{question}",
    # 입력변수로는 'question'을 이용한다.
    input_variables = ["question"],
    # 부분변수로는 'format_instructions'을 사용한다.
    partial_variables={"format_instructions":format_instructions},
)

In [18]:
model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model | output_parser

In [19]:
chain.invoke({"question":"대한민국의 수도는 어디인가요?"})

{'answer': '대한민국의 수도는 서울입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [20]:
chain.invoke({"대한민국의 수도는 어디인가요?"})

{'answer': '대한민국의 수도는 서울입니다.', 'source': 'https://www.korea.kr'}

`chain.stream` 메소드를 사용하여 질문에 대한 스트림 응답 받기. 하지만 제한적일수 있으므로, 가급적으로 invoke사용

In [21]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇안가요?"}):
    print(s)

{'answer': '세종대왕은 조선의 네 번째 왕으로, 한글(훈민정음)을 창제하여 한국어의 표기 체계를 혁신하였고, 과학, 음악, 농업 등 다양한 분야에서 많은 업적을 남겼습니다. 또한, 측우기와 같은 기상 관측 도구를 개발하여 농업 생산성을 높이는 데 기여하였습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
